# Computation on NumPy Arrays: Universal Functions

Up until now, we have been discussing some of the basic nuts and bolts of NumPy. In the next few chapters, we will dive into the reasons that NumPy is so important in the Python data science world: namely, because it provides an easy and flexible interface to optimize computation with arrays of data.

Computation on NumPy arrays can be very fast, or it can be very slow.
The key to making it fast is to use vectorized operations, generally implemented through NumPy's *universal functions* (ufuncs).
This chapter motivates the need for NumPy's ufuncs, which can be used to make repeated calculations on array elements much more efficient.
It then introduces many of the most common and useful arithmetic ufuncs available in the NumPy package.

## The Slowness of Loops

Python's default implementation (known as CPython) does some operations very slowly.
This is partly due to the dynamic, interpreted nature of the language; types are flexible, so sequences of operations cannot be compiled down to efficient machine code as in languages like C and Fortran.
Recently there have been various attempts to address this weakness: well-known examples are the [PyPy project](http://pypy.org/), a just-in-time compiled implementation of Python; the [Cython project](http://cython.org), which converts Python code to compilable C code; and the [Numba project](http://numba.pydata.org/), which converts snippets of Python code to fast LLVM bytecode.
Each of these has its strengths and weaknesses, but it is safe to say that none of the three approaches has yet surpassed the reach and popularity of the standard CPython engine.

The relative sluggishness of Python generally manifests itself in situations where many small operations are being repeated; for instance, looping over arrays to operate on each element.
For example, imagine we have an array of values and we'd like to compute the reciprocal of each.
A straightforward approach might look like this:

In [ ]:
import numpy as np
rng = np.random.default_rng(seed=1701)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = rng.integers(1, 10, size=5)
compute_reciprocals(values)

This implementation probably feels fairly natural to someone from, say, a C or Java background.
But if we measure the execution time of this code for a large input, we see that this operation is very slow—perhaps surprisingly so!
We'll benchmark this with IPython's `%timeit` magic:

In [ ]:
big_array = rng.integers(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

It takes several seconds to compute these million operations and to store the result!
When even cell phones have processing speeds measured in gigaflops (i.e., billions of numerical operations per second), this seems almost absurdly slow.
It turns out that the bottleneck here is not the operations themselves, but the type checking and function dispatches that CPython must do at each cycle of the loop.
Each time the reciprocal is computed, Python first examines the object's type and does a dynamic lookup of the correct function to use for that type.
If we were working in compiled code instead, this type specification would be known before the code executed and the result could be computed much more efficiently.

## Introducing Ufuncs

For many types of operations, NumPy provides a convenient interface into just this kind of statically typed, compiled routine. This is known as a *vectorized* operation.
For simple operations like the element-wise division here, vectorization is as simple as using Python arithmetic operators directly on the array object.
This vectorized approach is designed to push the loop into the compiled layer that underlies NumPy, leading to much faster execution.

Compare the results of the following two operations:

In [ ]:
print(compute_reciprocals(values))
print(1.0 / values)

Looking at the execution time for our big array, we see that it completes orders of magnitude faster than the Python loop:

In [ ]:
%timeit (1.0 / big_array)

Vectorized operations in NumPy are implemented via ufuncs, whose main purpose is to quickly execute repeated operations on values in NumPy arrays.
Ufuncs are extremely flexible—before we saw an operation between a scalar and an array, but we can also operate between two arrays:

In [ ]:
np.arange(5) / np.arange(1, 6)

And ufunc operations are not limited to one-dimensional arrays. They can act on multidimensional arrays as well:

In [ ]:
x = np.arange(9).reshape((3, 3))
2 ** x

Computations using vectorization through ufuncs are nearly always more efficient than their counterparts implemented using Python loops, especially as the arrays grow in size.
Any time you see such a loop in a NumPy script, you should consider whether it can be replaced with a vectorized expression.

## Exploring NumPy's Ufuncs

Ufuncs exist in two flavors: *unary ufuncs*, which operate on a single input, and *binary ufuncs*, which operate on two inputs.
We'll see examples of both these types of functions here.

### Array Arithmetic

NumPy's ufuncs feel very natural to use because they make use of Python's native arithmetic operators.
The standard addition, subtraction, multiplication, and division can all be used:

In [ ]:
x = np.arange(4)
print("x      =", x)
print("x + 5  =", x + 5)
print("x - 5  =", x - 5)
print("x * 2  =", x * 2)
print("x / 2  =", x / 2)
print("x // 2 =", x // 2)  # floor division

There is also a unary ufunc for negation, a `**` operator for exponentiation, and a `%` operator for modulus:

In [ ]:
print("-x     = ", -x)
print("x ** 2 = ", x ** 2)
print("x % 2  = ", x % 2)

In addition, these can be strung together however you wish, and the standard order of operations is respected:

In [ ]:
-(0.5*x + 1) ** 2

All of these arithmetic operations are simply convenient wrappers around specific ufuncs built into NumPy. For example, the `+` operator is a wrapper for the `add` ufunc:

In [ ]:
np.add(x, 2)

The following table lists the arithmetic operators implemented in NumPy:

| Operator    | Equivalent ufunc  | Description                         |
|-------------|-------------------|-------------------------------------|
|`+`          |`np.add`           |Addition (e.g., `1 + 1 = 2`)         |
|`-`          |`np.subtract`      |Subtraction (e.g., `3 - 2 = 1`)      |
|`-`          |`np.negative`      |Unary negation (e.g., `-2`)          |
|`*`          |`np.multiply`      |Multiplication (e.g., `2 * 3 = 6`)   |
|`/`          |`np.divide`        |Division (e.g., `3 / 2 = 1.5`)       |
|`//`         |`np.floor_divide`  |Floor division (e.g., `3 // 2 = 1`)  |
|`**`         |`np.power`         |Exponentiation (e.g., `2 ** 3 = 8`)  |
|`%`          |`np.mod`           |Modulus/remainder (e.g., `9 % 4 = 1`)|

Additionally, there are Boolean/bitwise operators; we will explore these in [Comparisons, Masks, and Boolean Logic](02.06-Boolean-Arrays-and-Masks.ipynb).

# Let's try this ourselves!

Let's pretend that `x` is a list of radii of spheres. The volume of a sphere is given by:
$$V=\frac{4}{3} π r^3$$

Let's calculate the volume of each sphere in `x` without using a loop. 

In [ ]:
π = np.pi 
sphere_volume = ...

Now can we time it?

In [ ]:
%%timeit # your sphere volume calculation here

### Absolute Value

Just as NumPy understands Python's built-in arithmetic operators, it also understands Python's built-in absolute value function:

In [ ]:
x = np.array([-2, -1, 0, 1, 2])
abs(x)

The corresponding NumPy ufunc is `np.absolute`, which is also available under the alias `np.abs`:

In [ ]:
np.absolute(x)

In [ ]:
np.abs(x)

### Exponents and Logarithms

Other common operations available in NumPy ufuncs are the exponentials:

In [ ]:
x = [1, 2, 3]
print("x   =", x)
print("e^x =", np.exp(x))
print("2^x =", np.exp2(x))
print("3^x =", np.power(3., x))

The inverse of the exponentials, the logarithms, are also available.
The basic `np.log` gives the natural logarithm; if you prefer to compute the base-2 logarithm or the base-10 logarithm, these are available as well:

In [ ]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

There are also some specialized versions that are useful for maintaining precision with very small input:

In [ ]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

When `x` is very small, these functions give more precise values than if the raw `np.log` or `np.exp` were to be used.

## Advanced Ufunc Features

Many NumPy users make use of ufuncs without ever learning their full set of features.
I'll outline a few specialized features of ufuncs here.

### Specifying Output

For large calculations, it is sometimes useful to be able to specify the array where the result of the calculation will be stored.
For all ufuncs, this can be done using the `out` argument of the function:

In [ ]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y)
print(y)

This can even be used with array views. For example, we can write the results of a computation to every other element of a specified array:

In [ ]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
print(y)

If we had instead written `y[::2] = 2 ** x`, this would have resulted in the creation of a temporary array to hold the results of `2 ** x`, followed by a second operation copying those values into the `y` array.
This doesn't make much of a difference for such a small computation, but for very large arrays the memory savings from careful use of the `out` argument can be significant.

## Ufuncs: Learning More

More information on universal functions (including the full list of available functions) can be found on the [NumPy](http://www.numpy.org) and [SciPy](http://www.scipy.org) documentation websites.

Recall that you can also access information directly from within IPython by importing the packages and using IPython's tab completion and help (`?`) functionality, as described in [Help and Documentation in IPython](01.01-Help-And-Documentation.ipynb).